<a href="https://colab.research.google.com/github/Kratosgado/audio-steganography/blob/from-claude/core_modules/lsb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install soundfile librosa

**Least Significant Bit (LSB) class**

In [ ]:
import numpy as np
import soundfile as sf
import librosa
from typing import Union, List

class AudioSteganography:
    def __init__(self, embedding_method='lsb'):
        """
        Initialize steganography module with selected embedding method

        Args:
            embedding_method (str): Method of embedding ('lsb', 'spread_spectrum')
        """
        self.embedding_method = embedding_method

    def _text_to_binary(self, message: str) -> List[int]:
        """
        Convert text message to binary representation

        Args:
            message (str): Input message to encode

        Returns:
            List[int]: Binary representation of message
        """
        return [int(bit) for char in message
                for bit in bin(ord(char))[2:].zfill(8)]

    def lsb_embed(self, audio_data: np.ndarray, message: str) -> np.ndarray:
        """
        Least Significant Bit embedding technique

        Args:
            audio_data (np.ndarray): Original audio numpy array
            message (str): Message to embed

        Returns:
            np.ndarray: Audio with embedded message
        """
        binary_message = self._text_to_binary(message)

        # Ensure message can be embedded
        if len(binary_message) > len(audio_data):
            raise ValueError("Message too large for audio")

        # Create copy of audio to modify
        stego_audio = audio_data.copy()

        # Embed each bit in LSB
        for i, bit in enumerate(binary_message):
            print("steg =  ", stego_audio[i], "bit =", bit)
            stego_audio[i] = (stego_audio[i] & 0xFE) | bit

        return stego_audio

    def lsb_extract(self, stego_audio: np.ndarray, message_length: int) -> str:
        """
        Extract hidden message using LSB technique

        Args:
            stego_audio (np.ndarray): Audio with embedded message
            message_length (int): Expected message length in characters

        Returns:
            str: Extracted message
        """
        # Extract least significant bits
        extracted_bits = [stego_audio[i] & 1 for i in range(message_length * 8)]

        # Convert bits back to characters
        binary_str = ''.join(map(str, extracted_bits))
        message = ''.join(
            chr(int(binary_str[i:i+8], 2))
            for i in range(0, len(binary_str), 8)
        )

        return message

def load_audio(file_path: str) -> np.ndarray:
    """
    Load audio file and convert to numpy array

    Args:
        file_path (str): Path to audio file

    Returns:
        np.ndarray: Audio data
    """
    audio_data, sample_rate = sf.read(file_path)
    return audio_data

def save_audio(audio_data: np.ndarray, file_path: str, sample_rate: int = 22050):
    """
    Save audio data to file

    Args:
        audio_data (np.ndarray): Audio numpy array
        file_path (str): Output file path
        sample_rate (int): Audio sample rate
    """
    sf.write(file_path, audio_data, sample_rate)



LSB:  functions test

In [ ]:
import wave

import struct  # For packing and unpacking the message length

def encode(input_file_path, output_file_path, secret_message):
    """
    Encodes a secret message into an audio file using basic LSB steganography with message length.

    :param input_file_path: Path to the input audio file
    :param output_file_path: Path to the output encoded audio file
    :param secret_message: The message to be encoded
    """
    try:
        print("Encoding starts...")
        audio = wave.open(input_file_path, mode="rb")
        frame_bytes = bytearray(list(audio.readframes(audio.getnframes())))

        print(f"Secret message: {secret_message}")
        # Convert the secret message to bits
        secret_message_bits = "".join(
            [bin(ord(i)).lstrip("0b").rjust(8, "0") for i in secret_message]
        )
        message_length = len(secret_message_bits)

        # Pack the length of the message into 4 bytes (32 bits)
        length_bytes = struct.pack(
            ">I", message_length
        )  # '>I' is big-endian unsigned int

        # Convert the length bytes to bits
        length_bits = "".join(
            [bin(byte).lstrip("0b").rjust(8, "0") for byte in length_bytes]
        )

        # Combine length bits and message bits
        full_bits = length_bits + secret_message_bits

        # Ensure the message fits into the frame bytes
        if len(full_bits) > len(frame_bytes):
            raise ValueError(
                "The secret message is too large to fit in the audio file."
            )

        # Encode the full bits into the frame bytes
        for i, bit in enumerate(full_bits):
            frame_bytes[i] = (frame_bytes[i] & 254) | int(bit)

        frame_modified = bytes(frame_bytes)

        # Write the modified bytes to the new audio file
        with wave.open(output_file_path, "wb") as new_audio:
            new_audio.setparams(audio.getparams())
            new_audio.writeframes(frame_modified)

        audio.close()
        print(f"Successfully encoded into {output_file_path}")
    except Exception as e:
        logger.error(f"Error during encoding: {e}")


def decode(input_file_path):
    """
    Decodes a secret message from an audio file using basic LSB steganography with message length.

    :param input_file_path: Path to the encoded audio file
    :return: The decoded secret message
    """
    try:
        print("Decoding starts...")
        audio = wave.open(input_file_path, mode="rb")
        frame_bytes = bytearray(list(audio.readframes(audio.getnframes())))

        # Extract the first 32 bits to determine the message length
        length_bits = "".join([str((frame_bytes[i] & 1)) for i in range(32)])
        message_length = struct.unpack(
            ">I", int(length_bits, 2).to_bytes(4, byteorder="big")
        )[0]

        print(f"Extracted message length: {message_length} bits")

        # Now extract the message bits using the extracted length
        if message_length > len(frame_bytes) * 8:
            raise ValueError(
                "The extracted message length is larger than the available audio data."
            )

        message_bits = "".join(
            [str((frame_bytes[i + 32] & 1)) for i in range(message_length)]
        )

        # Convert bits back to characters
        decoded_message = "".join(
            chr(int(message_bits[i : i + 8], 2)) for i in range(0, len(message_bits), 8)
        )

        print(f"Successfully decoded: {decoded_message}")
        audio.close()
        return decoded_message

    except Exception as e:
        print(f"Error during decoding: {e}")
        return None  # Example usage:


encode("sample_data/harvard.wav", "stego.wav", "secret123")
print(decode("stego.wav"))  # Output: Secret123

Encoding starts...
Secret message: secret123
Successfully encoded into stego.wav
Decoding starts...
Extracted message length: 72 bits
Successfully decoded: secret123
secret123


# AI-Assisted Robustness Analysis Module

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import librosa

class AudioRobustnessAnalyzer:
    def __init__(self):
        """
        Initialize robustness analysis module
        """
        self.model = self._build_detection_model()

    def _build_detection_model(self):
        """
        Build a simple neural network for steganalysis

        Returns:
            tf.keras.Model: Compiled detection model
        """
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(257,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        return model

    def signal_to_noise_ratio(self, original: np.ndarray, modified: np.ndarray) -> float:
        """
        Calculate Signal-to-Noise Ratio

        Args:
            original (np.ndarray): Original audio signal
            modified (np.ndarray): Modified audio signal

        Returns:
            float: SNR value
        """
        signal_power = np.mean(original**2)
        noise_power = np.mean((original - modified)**2)
        snr = 10 * np.log10(signal_power / noise_power)
        return snr

    def bit_error_rate(self, original: np.ndarray, modified: np.ndarray) -> float:
        """
        Calculate Bit Error Rate

        Args:
            original (np.ndarray): Original signal
            modified (np.ndarray): Modified signal

        Returns:
            float: Bit Error Rate
        """
        # Convert signals to binary representation
        original_bits = np.unpackbits(original.astype(np.uint8))
        modified_bits = np.unpackbits(modified.astype(np.uint8))

        # Compute bit-wise differences
        errors = np.sum(original_bits != modified_bits)
        total_bits = len(original_bits)

        return errors / total_bits

    def train_steganalysis_model(self, cover_samples, stego_samples):
        """
        Train a simple steganalysis model

        Args:
            cover_samples (np.ndarray): Cover audio samples
            stego_samples (np.ndarray): Stego audio samples
        """
        # Feature extraction
        cover_features = np.array([
            librosa.feature.spectral_contrast(y=sample).mean()
            for sample in cover_samples
        ])
        stego_features = np.array([
            librosa.feature.spectral_contrast(y=sample).mean()
            for sample in stego_samples
        ])

        # Prepare training data
        X = np.concatenate([cover_features, stego_features])
        y = np.concatenate([
            np.zeros(len(cover_samples)),
            np.ones(len(stego_samples))
        ])

        # Train model
        self.model.fit(X, y, epochs=50, validation_split=0.2)

    def detect_steganography(self, audio_sample: np.ndarray) -> float:
        """
        Predict probability of steganography

        Args:
            audio_sample (np.ndarray): Audio sample to analyze

        Returns:
            float: Probability of steganography
        """
        features = librosa.feature.spectral_contrast(y=audio_sample).mean()
        return self.model.predict(features.reshape(1, -1))[0][0]

# Example Usage
if __name__ == "__main__":
    # Load sample audio
    original_audio = librosa.load('sample_data/harvard.wav')[0]
    stego_audio = librosa.load('stego.wav')[0]

    # Initialize analyzer
    analyzer = AudioRobustnessAnalyzer()

    # Performance Metrics
    snr = analyzer.signal_to_noise_ratio(original_audio, stego_audio)
    ber = analyzer.bit_error_rate(original_audio, stego_audio)

    print(f"Signal-to-Noise Ratio: {snr:.2f} dB")
    print(f"Bit Error Rate: {ber:.4f}")

    # Steganography Detection
    detection_prob = analyzer.detect_steganography(stego_audio)
    print(f"Steganography Detection Probability: {detection_prob:.2%}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Signal-to-Noise Ratio: 68.83 dB
Bit Error Rate: 0.0000


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 257, but received input with shape (1, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 1), dtype=float32)
  • training=False
  • mask=None

# Attack Simulation Module

In [ ]:
import numpy as np
import librosa
import soundfile as sf

class AudioAttackSimulator:
    def __init__(self, noise_level: float = 0.01):
        """
        Initialize attack simulator

        Args:
            noise_level (float): Intensity of noise injection
        """
        self.noise_level = noise_level

    def gaussian_noise_attack(self, audio_data: np.ndarray) -> np.ndarray:
        """
        Simulate Gaussian noise attack

        Args:
            audio_data (np.ndarray): Input audio data

        Returns:
            np.ndarray: Audio with added Gaussian noise
        """
        noise = np.random.normal(
            0,
            self.noise_level * np.max(np.abs(audio_data)),
            audio_data.shape
        )
        return audio_data + noise

    def compression_attack(self, audio_path: str, quality: int = 3) -> np.ndarray:
        """
        Simulate lossy compression attack

        Args:
            audio_path (str): Path to input audio file
            quality (int): Compression quality (1-9, lower is more aggressive)

        Returns:
            np.ndarray: Compressed audio data
        """
        from scipy.io import wavfile
        import subprocess
        import os

        # Temporary files for compression simulation
        temp_compressed = 'temp_compressed.mp3'
        temp_decompressed = 'temp_decompressed.wav'

        # Use FFmpeg for compression simulation
        subprocess.call([
            'ffmpeg',
            '-i', audio_path,
            '-q:a', str(quality),
            temp_compressed
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Decompress back to WAV
        subprocess.call([
            'ffmpeg',
            '-i', temp_compressed,
            temp_decompressed
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Load decompressed audio
        compressed_audio, _ = librosa.load(temp_decompressed)

        # Clean up temporary files
        os.remove(temp_compressed)
        os.remove(temp_decompressed)

        return compressed_audio

    def frequency_filtering(self, audio_data: np.ndarray,
                             low_cutoff: float = 100,
                             high_cutoff: float = 10000) -> np.ndarray:
        """
        Apply frequency domain filtering

        Args:
            audio_data (np.ndarray): Input audio data
            low_cutoff (float): Low frequency cutoff
            high_cutoff (float): High frequency cutoff

        Returns:
            np.ndarray: Filtered audio data
        """
        # Compute Short-time Fourier Transform
        stft = librosa.stft(audio_data)

        # Apply frequency mask
        mask = np.zeros_like(stft, dtype=bool)
        freqs = librosa.fft_frequencies()
        mask[(freqs >= low_cutoff) & (freqs <= high_cutoff)] = True

        filtered_stft = stft * mask

        # Convert back to time domain
        return librosa.istft(filtered_stft)

# Example Usage
if __name__ == "__main__":
    # Load stego audio
    stego_audio, sr = librosa.load('stego_audio.wav')

    # Initialize attack simulator
    attacker = AudioAttackSimulator(noise_level=0.05)

    # Simulate different attacks
    noisy_audio = attacker.gaussian_noise_attack(stego_audio)
    compressed_audio = attacker.compression_attack('stego_audio.wav')
    filtered_audio = attacker.frequency_filtering(stego_audio)

    # Save attacked audio samples
    sf.write('noisy_audio.wav', noisy_audio, sr)
    sf.write('compressed_audio.wav', compressed_audio, sr)
    sf.write('filtered_audio.wav', filtered_audio, sr)